In [8]:
import pandas as pd
import string
from datasets import Dataset

df = pd.read_csv('./RAW_recipes.csv')
df['text'] = df['steps']
df = df[['text']]
df['text'] = df['text'].str.replace(f"[{string.punctuation}]", "", regex=True)

In [3]:
import pandas as pd
import string
from datasets import Dataset

df = pd.read_csv('./RAW_recipes.csv')
df['text'] = df['steps']
df = df[['text']]
df['text'] = df['text'].str.replace(f"[{string.punctuation}]", "", regex=True)

dataset = Dataset.from_pandas(df)

# Split the dataset into train (you can adjust the train/test split ratio)
train_dataset = dataset.train_test_split(test_size=0.2)["train"]

# Verify the train dataset
print(train_dataset)

Dataset({
    features: ['text'],
    num_rows: 185309
})


In [3]:
from datasets import load_dataset

dataset_name = "mlabonne/guanaco-llama2-1k"
dataset = load_dataset(dataset_name,split = "train")
dataset

/raid/80024217/anaconda3/envs/py10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['text'],
    num_rows: 1000
})

In [4]:
import os
import torch
import pandas as pd
import string
from datasets import Dataset
from datasets import load_dataset
from transformers import ( 
    AutoTokenizer, 
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    
    )
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

model_name = "meta-llama/Llama-2-7b-chat-hf"
new_model = "Llama-2-7b-chat-finetune-qlora"

device_map = {"":0}

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

prompt = "How can I learn to optimize my webpage for search engines?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

ModuleNotFoundError: No module named 'trl'